In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import malaya_speech
import malaya_speech.config
import malaya_speech.train.model.quartznet as quartznet
import tensorflow as tf


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [5]:
config = malaya_speech.config.ctc_featurizer_config
featurizer = malaya_speech.tf_featurization.STTFeaturizer(config)

In [11]:
i = tf.placeholder(tf.float32, [None])
v = featurizer.vectorize(i)
length_v = tf.cast(tf.shape(v)[0], tf.int32)
length_v = tf.expand_dims(length_v, 0)
v = tf.expand_dims(v[:,:,0], 0)
v, length_v

(<tf.Tensor 'ExpandDims_11:0' shape=(1, ?, 80) dtype=float32>,
 <tf.Tensor 'ExpandDims_10:0' shape=(1,) dtype=int32>)

In [12]:
model = quartznet.Model(v, length_v)
model.logits

Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use `tf.keras.layers.Conv1D` instead.


{'outputs': <tf.Tensor 'w2l_encoder_1/Relu_77:0' shape=(1, ?, 1024) dtype=float32>,
 'src_length': <tf.Tensor 'w2l_encoder_1/floordiv_154:0' shape=(1,) dtype=int32>}

In [13]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [15]:
y, sr = malaya_speech.load('../speech/example-speaker/husein-zolkepli.wav', sr = 16000)
y1, sr = malaya_speech.load('../speech/example-speaker/shafiqah-idayu.wav', sr = 16000)
len(y), len(y1)

(90090, 56298)

In [18]:
l = sess.run(model.logits, feed_dict = {i: y})
l

{'outputs': array([[[0.6258866 , 0.        , 1.3304489 , ..., 0.65230155,
          0.27502653, 0.37272736],
         [0.3395493 , 0.0743845 , 0.47811994, ..., 0.        ,
          0.74452716, 0.8826371 ],
         [1.2375557 , 0.81549716, 0.8892278 , ..., 0.        ,
          0.02212776, 0.        ],
         ...,
         [0.        , 0.70464855, 0.        , ..., 0.6296963 ,
          0.4864693 , 0.        ],
         [0.        , 0.        , 1.4843035 , ..., 0.17836735,
          0.7116283 , 0.45895416],
         [0.        , 0.22543716, 0.27976868, ..., 0.        ,
          0.5504492 , 0.        ]]], dtype=float32),
 'src_length': array([281], dtype=int32)}

In [20]:
l = sess.run(model.logits, feed_dict = {i: y1})
l

{'outputs': array([[[0.        , 1.6043066 , 0.75907594, ..., 0.        ,
          0.712527  , 0.        ],
         [0.05429816, 0.        , 0.        , ..., 0.40704766,
          1.1729138 , 0.56751335],
         [0.        , 0.        , 0.84713185, ..., 1.5323976 ,
          0.44171074, 1.3103052 ],
         ...,
         [0.36601436, 0.8669824 , 0.13385634, ..., 0.        ,
          0.82642144, 0.        ],
         [0.        , 0.        , 0.80695796, ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.6996588 , 0.71140087, ..., 0.2167407 ,
          0.        , 0.01738247]]], dtype=float32),
 'src_length': array([175], dtype=int32)}